In [1]:
import numpy as np
from datetime import datetime
from ann_utils import *
from ann_structure import *

%load_ext autoreload
%autoreload 2

# Implement the NN:

In [22]:
def L_layer_model(X, Y, layer_dims, learning_rate =0.05, num_iterations=60, print_cost=True):
    
    costs = []
    
    parameters = init_params(layer_dims)
    
    for i in range(num_iterations):
    
        # forward pass:
        AL, caches = L_forward(X, parameters)

        # calculate the cost:
        #S = softmax(AL)
        cost = cost_total(AL, Y)
        costs.append(cost)

        # backward pass:
        grads = L_backward(AL, Y, caches)
        
        ### THIS IS WHERE GRADIENT CHECKING WOULD BE
        
        # update parameters:
        parameters = update_parameters(parameters, grads, learning_rate)
        
        # print the cost:
        if print_cost and any([(i+1)%10==0, i == 0]):
            print('Cost after ' + str(i + 1) + ' iterations is ' + str(cost) + ' at ' + str(datetime.now()))
    
    return parameters

In [66]:
def predict(X, parameters):
    
    L = len(parameters) // 2
    m = X.shape[1]
    
    AL, caches = L_forward(X, parameters)
    
    S = AL.copy()
    max_prob = np.argmax(S, axis=0)
    
    #print(max_prob.shape)
    
    for i in range(m):
        S[:, i] = 0
        S[max_prob[i], i] = 1
    
    return S

In [67]:
def accuracy(Y_hat, Y):
    
    assert(Y_hat.shape == Y.shape)
    
    m = Y.shape[1] # number of data entries
    
    w = 0
    c = 0
    
    A = Y_hat.T
    B = Y.T
    assert(A.shape == B.shape)
    for i in range(Y.shape[1]):
        if np.all(A[i] == B[i]):
            c += 1
        else:
            w += 1
    
    return c/m

# Train and Test the NN:

In [88]:
X = np.load('data/covtype_train_x.npy')
Y = np.load('data/covtype_train_y.npy')

In [89]:
def standard_score_rescaling(X):
    '''
    Rescales the features to their Z-score
    '''
    m = X.shape[1]
    mu = (1 / m) * np.sum(X, axis=1, keepdims=True)
    
    X = X - mu
    
    sigma = (1 / m) * np.sum(X**2, axis=1, keepdims=True)

    X = X / sigma
    
    return X

def feature_rescaling(X):
    '''
    Rescales the features to the interval [0, 1]
    '''
    mn = np.min(X, axis=1, keepdims=True)
    mx = np.max(X, axis=1, keepdims=True)
    
    X = (X - mn) / (mx - mn)
    
    return X

In [85]:
X = feature_rescaling(X)

In [86]:
layer_dims = (54, 70, 50, 30, 20, 7)

In [ ]:
params = L_layer_model(X, Y, layer_dims, learning_rate =0.1, num_iterations=70)

In [77]:
out = predict(X, params)
out = out.squeeze()

In [ ]:
accuracy(out, Y)